<a href="https://colab.research.google.com/github/NafishaNower/Dermatological-Disease-Detection-Using-Image-Processing-And-Deep-Learning/blob/main/Dermatological_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unet

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K

# Assuming you have 7 types of diseases
num_classes = 7

# Set the paths to your image and mask folders
image_folder = "/content/drive/MyDrive/final 438/disease/Original/Final dataset"
mask_folder = "/content/drive/MyDrive/final 438/disease/Mask/Mask"

# Step 1: Load and Preprocess Data
image_filenames = os.listdir(image_folder)
mask_filenames = os.listdir(mask_folder)

image_filenames.sort()
mask_filenames.sort()

images = []
masks = []

for img_filename, mask_filename in zip(image_filenames, mask_filenames):
    img_path = os.path.join(image_folder, img_filename)
    mask_path = os.path.join(mask_folder, mask_filename)

    img = load_img(img_path, target_size=(128, 128))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    images.append(img_array)

    # Load and preprocess the mask (assuming it's one-hot encoded)
    mask = load_img(mask_path, target_size=(128, 128), color_mode='grayscale')
    mask_array = img_to_array(mask) / 255.0  # Normalize pixel values to [0, 1]
    masks.append(mask_array)

# Convert lists to NumPy arrays
images = np.array(images)
masks = np.array(masks)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train_categorical = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_categorical = tf.keras.utils.to_categorical(y_test, num_classes)

# Step 2: Build the Modified U-Net Model for 7 Classes
def unet_model(input_size=(128, 128, 3), num_classes=num_classes):
    inputs = tf.keras.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(conv5)
    up6 = layers.concatenate([up6, conv4], axis=-1)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    up7 = layers.concatenate([up7, conv3], axis=-1)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    up8 = layers.concatenate([up8, conv2], axis=-1)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    up9 = layers.concatenate([up9, conv1], axis=-1)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layer with softmax activation for multi-class classification
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(conv9)

    model = models.Model(inputs=inputs, outputs=outputs)

    return model


# Create an instance of the U-Net model
model = unet_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 3: Training
history = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_test, y_test_categorical))

# Step 4: Evaluation
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test Accuracy: {test_accuracy}')

# Save the model architecture as JSON
model_json = model.to_json()
with open("unet_skin_disease_classification_model.json", "w") as json_file:
    json_file.write(model_json)

# Save the weights
model.save_weights("unet_skin_disease_classification_weights.h5")

# Save the entire model (architecture + weights)
model.save("unet_skin_disease_classification_model_complete.h5")

# Plot training and validation loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy curves
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# ... (Previous code remains unchanged)

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss Curves')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy Curves')
plt.show()

# Confusion Matrix
plt.figure(figsize=(8, 6))
conf_matrix = confusion_matrix(np.argmax(y_test_categorical, axis=1), np.argmax(y_pred, axis=1))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(np.argmax(y_test_categorical, axis=1), np.argmax(y_pred, axis=1)))

# Visualize CNN Model Architecture
tf.keras.utils.plot_model(model, to_file='unet_skin_disease_classification_model.png', show_shapes=True)

# Calculate precision, recall, and F1-score
precision = precision_score(np.argmax(y_test_categorical, axis=1), np.argmax(y_pred, axis=1), average='weighted')
recall = recall_score(np.argmax(y_test_categorical, axis=1), np.argmax(y_pred, axis=1), average='weighted')
f1 = f1_score(np.argmax(y_test_categorical, axis=1), np.argmax(y_pred, axis=1), average='weighted')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')

from IPython.display import Image

Image("model_diagram.png")

# Visualize CNN Model Architecture
tf.keras.utils.plot_model(model, to_file='unet_skin_disease_classification_model.png', show_shapes=True)


VGG19

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg19 import VGG19
from skimage import exposure  # Add this line for the exposure module

# Assuming you have 7 types of diseases
num_classes = 7

# Set the paths to your image and mask folders
image_folder = "/content/drive/MyDrive/final 438/disease/Original/Final dataset"
mask_folder = "/content/drive/MyDrive/final 438/disease/Mask/Mask"

# ... (Previous data loading and preprocessing code remains unchanged) ...
# Step 1: Load and Preprocess Data
image_filenames = os.listdir(image_folder)
mask_filenames = os.listdir(mask_folder)

image_filenames.sort()
mask_filenames.sort()

images = []
masks = []
labels = []

for img_filename, mask_filename in zip(image_filenames, mask_filenames):
    img_path = os.path.join(image_folder, img_filename)
    mask_path = os.path.join(mask_folder, mask_filename)

    # Load image and apply histogram equalization
    img = load_img(img_path, target_size=(224, 224), color_mode='grayscale')
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = exposure.equalize_hist(img_array.squeeze())  # Apply histogram equalization
    img_array = np.expand_dims(img_array, axis=-1)  # Add back the channel dimension
    img_array_rgb = np.concatenate([img_array, img_array, img_array], axis=-1)  # Create pseudo-RGB image
    images.append(img_array_rgb)

    # Extract label from the filename
    label_parts = img_filename.split('-')[-1].split('.')[0].split('_')
    label = "".join(label_parts[:-1]).replace(" ", "")
    labels.append(label)

    # Load and preprocess the mask (assuming it's one-hot encoded)
    mask = load_img(mask_path, target_size=(224, 224), color_mode='grayscale')
    mask_array = img_to_array(mask) / 255.0  # Normalize pixel values to [0, 1]
    masks.append(mask_array)

# Convert lists to NumPy arrays
images = np.array(images)
masks = np.array(masks)
labels = np.array(labels)

# Convert labels to categorical
label_encoder = LabelEncoder()
y_categorical = tf.keras.utils.to_categorical(label_encoder.fit_transform(labels), num_classes)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, y_categorical, test_size=0.2, random_state=42)

# Rest of the code up to the model creation remains unchanged...

# Step 2: Build the Modified VGG19 Model for 7 Classes
def vgg19_model(input_size=(224, 224, 3), num_classes=num_classes):
    base_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=input_size)

    for layer in base_model.layers:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Create an instance of the VGG19 model
model = vgg19_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ... (Callbacks and Training code remains unchanged) ...
# Learning Rate Scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.9 ** epoch)
# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Step 3: Training
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[lr_scheduler, early_stopping])

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss Curves')
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy Curves')
plt.show()

# Confusion Matrix
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
class_report = classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
print("Classification Report:")
print(class_report)

# Calculate precision, recall, and F1-score
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')

# Visualize CNN Model Architecture
tf.keras.utils.plot_model(model, to_file='vgg19_skin_disease_classification_model.png', show_shapes=True)


MobileNetV2

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have 7 types of diseases
num_classes = 7

# Set the paths to your image and mask folders
image_folder = "/content/drive/MyDrive/final 438/disease/Original/Final dataset"
mask_folder = "/content/drive/MyDrive/final 438/disease/Mask/Mask"

# Step 1: Load and Preprocess Data
image_filenames = os.listdir(image_folder)
mask_filenames = os.listdir(mask_folder)

image_filenames.sort()
mask_filenames.sort()

images = []
masks = []
labels = []

for img_filename, mask_filename in zip(image_filenames, mask_filenames):
    img_path = os.path.join(image_folder, img_filename)
    mask_path = os.path.join(mask_folder, mask_filename)

    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    images.append(img_array)

    # Extract label from the filename
    label_parts = img_filename.split('-')[-1].split('.')[0].split('_')
    label = "".join(label_parts[:-1]).replace(" ", "")
    labels.append(label)

    # Load and preprocess the mask (assuming it's one-hot encoded)
    mask = load_img(mask_path, target_size=(224, 224), color_mode='grayscale')
    mask_array = img_to_array(mask) / 255.0  # Normalize pixel values to [0, 1]
    masks.append(mask_array)

# Convert lists to NumPy arrays
images = np.array(images)
masks = np.array(masks)
labels = np.array(labels)

# Convert labels to categorical
label_encoder = LabelEncoder()
y_categorical = tf.keras.utils.to_categorical(label_encoder.fit_transform(labels), num_classes)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, y_categorical, test_size=0.2, random_state=42)

# Step 2: Build the Modified MobileNetV2 Model for 7 Classes
base_model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save the model architecture diagram to a file
plot_model(model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)

# Step 3: Training
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Step 4: Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

# Additional evaluation metrics
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Save the trained model
model.save('mobilenetv2_skin_disease_classification_model.h5')

# Plotting Loss and Accuracy Curves
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Loss curve
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy curve
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

# F1 Score Calculation and Curve
from sklearn.metrics import f1_score

f1_scores = []
for epoch in range(len(history.history['loss'])):
    f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')
    f1_scores.append(f1)

plt.plot(f1_scores, label='Weighted F1 Score')
plt.title('F1 Score Curve')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()
plt.show()

# AUC-ROC Curve
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(8, 6))

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC-ROC Curve')
plt.legend()
plt.show()

# Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report

conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
print("Classification Report:")
print(class_report)

# Grad-CAM Heatmap
def get_gradcam_heatmap(model, img_array, class_index, layer_name='out_relu'):
    # Define a model that maps the input image to the activations of the specified layer and output layer
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    # Get the convolutional layer output and model prediction for the given input
    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img_array)
        loss = predictions[:, class_index]

    # Calculate gradients of the loss with respect to the output feature map of the selected layer
    grads = tape.gradient(loss, conv_output)[0]

    # Global average pooling of gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    #
